<a href="https://colab.research.google.com/github/mohsenh17/jaxLearning/blob/main/flax/riceTypes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import kagglehub
path = kagglehub.dataset_download("mssmartypants/rice-type-classification")

In [27]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np


In [21]:
df = pd.read_csv("dataset/riceClassification.csv")
df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [37]:
class CustomImageDataset(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        completeDF = pd.read_csv(dataset, index_col=0)
        self.labels = pd.DataFrame(completeDF['Class'])
        self.features = pd.DataFrame(completeDF.drop('Class', axis=1))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = np.array(self.features.iloc[idx, :])
        label = self.labels.iloc[idx, 0]
        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)
        return features, label

In [39]:
dataset = CustomImageDataset(
    dataset="dataset/riceClassification.csv"
)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

for features, labels in data_loader:
    print("Batch of features has shape: ",features.shape)
    print("Batch of labels has shape: ", labels.shape)
    print(features)
    print(labels)
    break

Batch of features has shape:  torch.Size([4, 10])
Batch of labels has shape:  torch.Size([4])
tensor([[9.2550e+03, 1.6128e+02, 7.3432e+01, 8.9033e-01, 9.3870e+03, 1.0855e+02,
         7.7074e-01, 3.8315e+02, 7.9223e-01, 2.1963e+00],
        [9.7310e+03, 1.6482e+02, 7.6050e+01, 8.8719e-01, 1.0010e+04, 1.1131e+02,
         7.2371e-01, 3.9671e+02, 7.7699e-01, 2.1673e+00],
        [9.1920e+03, 1.6547e+02, 7.1622e+01, 9.0147e-01, 9.3680e+03, 1.0818e+02,
         5.8900e-01, 3.8885e+02, 7.6393e-01, 2.3103e+00],
        [6.7640e+03, 1.6603e+02, 5.2837e+01, 9.4801e-01, 6.9830e+03, 9.2802e+01,
         4.7972e-01, 3.6618e+02, 6.3389e-01, 3.1422e+00]], dtype=torch.float64)
tensor([0, 0, 0, 1])
